In [ ]:
# %pip install python-gnupg
# %pip install azure-storage-file-datalake
# %restart_python

In [ ]:
import gnupg
import subprocess
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO
from datetime import datetime

In [ ]:
def get_datalake_client(account_name, account_key):
    service_client = DataLakeServiceClient(
        account_url=f"https://{account_name}.dfs.core.windows.net", credential=account_key
    )
    return service_client

In [ ]:
service_client = get_datalake_client("qambusa", Access_key)

In [ ]:
def download_file_from_datalake(service_client, filesystem_name, directory_name, file_name):
    filesystem_client = service_client.get_file_system_client(filesystem_name)
    directory_client = filesystem_client.get_directory_client(directory_name)
    file_client = directory_client.get_file_client(file_name)
    
    download = file_client.download_file()
    file_data = download.readall()
    print(f"Downloaded {file_name} from Azure Data Lake")
    return file_data

In [ ]:
public_key_data = download_file_from_datalake(service_client, "clicmotion", "test/encrypted_keys", "public_key.asc")
print(public_key_data)

In [ ]:
def import_gpg_public_key(public_key_data):
    process = subprocess.run(["gpg", "--import"], input=public_key_data, capture_output=True)
    print(f"process: {process}")
    if process.returncode != 0:
        print("Error importing public key:", process.stderr.decode())
        raise Exception("GPG key import failed")
    print("Public key imported successfully")

In [ ]:
import_gpg_public_key(public_key_data)

In [ ]:
def get_gpg_recipient_id():
    # List the imported GPG keys
    process = subprocess.run(["gpg", "--list-keys", "--with-colons"], capture_output=True, text=True)
    if process.returncode != 0:
        print("Error listing GPG keys:", process.stderr.decode())
        raise Exception("GPG key listing failed")
        # Find the key ID from the list
    print(process.stdout.splitlines())
    keys = process.stdout.splitlines()
    for line in keys:
        if line.startswith("pub"):
            # Extract the key ID from the line (key ID is the part after the key type)
            key_id = line.split(":")[4]
            print(f"Found GPG key ID: {key_id}")
            return key_id
    raise Exception("No valid GPG key found")

In [ ]:
recipient_id = get_gpg_recipient_id()

In [ ]:
file_data = download_file_from_datalake(service_client, "clicmotion", "test", "dummy_file_for_encryption.txt")

In [ ]:
def gpg_encrypt_data(input_data, recipient_id):
    process = subprocess.Popen(
        ["gpg", "--encrypt", "--batch", "--yes", "--recipient", recipient_id, "--no-tty", "--trust-model", "always"],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    encrypted_data, error = process.communicate(input=input_data)
    
    if process.returncode != 0:
        print("Error during GPG encryption:", error.decode())
        raise Exception("GPG encryption failed")
    
    print("Data encrypted successfully")
    return encrypted_data

In [ ]:
encrypted_data = gpg_encrypt_data(file_data, recipient_id)

In [ ]:
def upload_encrypted_data_to_datalake(service_client, filesystem_name, directory_name, file_name, encrypted_data):
    filesystem_client = service_client.get_file_system_client(filesystem_name)
    directory_client = filesystem_client.get_directory_client(directory_name)
    file_client = directory_client.get_file_client(file_name)
    
    file_client.upload_data(encrypted_data, overwrite=True)
    print(f"Encrypted file uploaded as {file_name}")

In [ ]:
upload_encrypted_data_to_datalake(service_client, "clicmotion", "test", "encrypted_data.txt.gpg", encrypted_data)